# Módulo 7 - Actividad 3:
# Reconstrucción de imágenes y eliminación de ruido usando Autoencoders

## Objetivo
Comparar el desempeño de ambos modelos y analizar visualmente los resultados obtenidos.

**Datasets utilizados:**  
`MNIST`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

> Para facilitar la comprensión del código, cada etapa del proceso está modularizada en funciones, integrando la carga de datos, creación y entrenamiento de modelos, y visualización de resultados. Esto permite entender el papel específico de cada bloque dentro del flujo general de autoencoders.

1. **Carga y preprocesamiento de datos:**
    - Se carga el dataset **MNIST**, normalizando las imágenes y aplanándolas para adecuarlas a la entrada del autoencoder.

2. **Construcción y entrenamiento del autoencoder básico:**
    - Se crea un autoencoder con arquitectura 784 - 128 - 64 - 128 - 784.
    - El modelo se entrena con imágenes limpias para aprender la reconstrucción directa.

3. **Generación de ruido y entrenamiento del denoising autoencoder:**
    - Se generan versiones ruidosas del dataset aplicando ruido gaussiano y manteniendo los valores en el rango [0, 1].
    - Se entrena el mismo modelo de autoencoder para que aprenda a reconstruir imágenes limpias a partir de entradas ruidosas.

4. **Comparación y visualización de pérdidas y reconstrucciones entre modelos:**
    - Se visualiza la comparación de la evolución de la pérdida de reconstrucción durante el entrenamiento del Autoencoder básico y del Denoising Autoencoder.
    - Se visualizan las imágenes originales, reconstruidas por Autoencoder básico, ruidosas y las reconstruidas para comprobar la eficacia del denoising autoencoder.
    - Se construye una tabla comparativa con las pérdidas y pérdidas de validación de ambos modelos a lo largo de las épocas.

---

# 2. Configuración del entorno

---

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.gridspec import GridSpec
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from IPython.display import display

# Establecer seeds
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos.

- **`cargar_datos()`** 
Carga y preprocesa el dataset MNIST en formato vectorizado.

---

In [ ]:
def cargar_datos():
    """
    Carga el dataset MNIST, normaliza los valores a [0, 1] y aplana las imágenes.

    Returns:
        tuple: (x_train, x_test) con las imágenes procesadas en formato (num_samples, 784).
    """
    (x_train, _), (x_test, _) = mnist.load_data()
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = x_train.reshape((len(x_train), -1))
    x_test = x_test.reshape((len(x_test), -1))
    return x_train, x_test

**Bloque 2:** Autoencoder básico

- **`construir_autoencoder()`** 
Crea el modelo de autoencoder básico para compresión y reconstrucción de imágenes.

- **`entrenar_autoencoder()`** 
Entrena el autoencoder básico usando imágenes limpias como entrada y salida.

---

##### `Decisiones de diseño`

##### **Justificación de la construcción del Autoencoder**

- El autoencoder que se construyó se basa en una arquitectura típica con un codificador y un decodificador. El codificador reduce la dimensión de entrada (en este caso, 784 píxeles de imágenes MNIST aplanadas) a un espacio latente más pequeño, pasando por capas con 128 y luego 64 neuronas. Esto comprime la información relevante. El decodificador realiza el proceso inverso, expandiendo nuevamente la representación comprimida para reconstruir la imagen original. Esta estructura permite que la red aprenda una representación compacta pero informativa de los datos.

- Para las capas ocultas se utilizó la función de activación ReLU (Rectified Linear Unit). ReLU es una función no lineal que transforma la entrada dejando pasar los valores positivos y poniendo a cero los negativos. Esto es importante porque la no linealidad permite al modelo capturar patrones complejos en los datos, mientras que su simplicidad computacional facilita el entrenamiento eficiente y reduce problemas como el desvanecimiento del gradiente en redes profundas. Por ello, es común y efectivo usar ReLU en las capas intermedias tanto del codificador como del decodificador.

- En la capa de salida, sin embargo, se usó una activación sigmoide que mapea los valores a un rango entre 0 y 1. Esto es coherente con el preprocesamiento que hicimos al normalizar las imágenes MNIST a ese mismo rango. De esta forma, la red puede generar una reconstrucción que se interpreta como probabilidades o intensidades normalizadas de píxeles, lo que facilita que la salida tenga sentido visual y numérico para imágenes.

- En cuanto a la función de pérdida, se eligió binary crossentropy. Aunque este criterio se asocia tradicionalmente con tareas de clasificación binaria, resulta muy efectivo para medir la diferencia entre imágenes cuyos píxeles son valores en [0,1]. La pérdida se calcula píxel a píxel, comparando la probabilidad de píxel reconstruido con el valor original. Esto penaliza fuertemente las diferencias en cada píxel y suele ayudar a que el autoencoder aprenda a reconstruir detalles finos de las imágenes, algo que a veces la pérdida cuadrática media (MSE) no logra tan eficientemente.

- Finalmente, el uso del optimizador Adam permite un entrenamiento más rápido y estable. Adam adapta el aprendizaje de cada peso de forma individual combinando el método de momento y ajuste adaptativo del learning rate, facilitando la convergencia del modelo con menos ajustes manuales.

---

In [ ]:
def construir_autoencoder(input_dim):
    """
    Construye y compila un autoencoder básico con arquitectura 784 → 128 → 64 → 128 → 784.

    Args:
        input_dim (int): Dimensión de entrada de los datos (por ejemplo, 784 para MNIST).

    Returns:
        Model: Modelo Keras compilado del autoencoder.
    """
    input_img = Input(shape=(input_dim,))
    # Codificador
    encoded = Dense(128, activation='relu')(input_img)
    encoded = Dense(64, activation='relu')(encoded)
    # Decodificador
    decoded = Dense(128, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    # Modelo completo
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')
    return autoencoder

def entrenar_autoencoder(autoencoder, x_train, x_test, epochs=20, batch_size=256):
    """
    Entrena un autoencoder con imágenes limpias como entrada y salida.

    Args:
        autoencoder (Model): Modelo Keras del autoencoder.
        x_train (ndarray): Datos de entrenamiento.
        x_test (ndarray): Datos de validación.
        epochs (int, opcional): Número de épocas de entrenamiento. Default 20.
        batch_size (int, opcional): Tamaño de lote. Default 256.

    Returns:
        History: Objeto con el historial de entrenamiento.
    """
    history_A = autoencoder.fit(
        x_train, x_train,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=True,
        validation_data=(x_test, x_test),
        verbose=0
    )
    return history_A

**Bloque 3:** Autoencoder para denoising

- **`generar_ruido()`** 
Genera versiones ruidosas del dataset manteniendo valores en [0, 1].

- **`entrenar_denoising_autoencoder()`** 
Entrena un autoencoder para reconstruir imágenes limpias a partir de entradas ruidosas.

---

##### `Decisiones de diseño`

##### **Justificación del Denoising Autoencoder:**

- En esta parte del trabajo se utilizó ruido gaussiano para generar versiones ruidosas de las imágenes originales. El propósito de esto es entrenar un denoising autoencoder, que es un modelo diseñado para aprender a limpiar o eliminar ruido de los datos de entrada. Al presentar al modelo imágenes con ruido como entrada y las imágenes originales limpias como salida esperada, se fuerza al autoencoder a aprender representaciones más robustas y generalizables. Esto permite que el modelo no solo reconstruya las imágenes originales, sino que también aprenda a filtrar y corregir perturbaciones, mejorando su capacidad para manejar datos ruidosos en aplicaciones reales.

- Respecto a la arquitectura, en esta etapa no se vuelve a definir ni modificar la estructura del autoencoder (es decir, no se repite explícitamente la reducción dimensional de 784 a 64 y la expansión inversa). Esto se debe a que se reutiliza el mismo modelo previamente construido en la parte anterior de autoencoder. El denoising autoencoder comparte la misma arquitectura porque la tarea fundamental sigue siendo la reconstrucción, pero ahora con una dificultad mayor: limpiar el ruido. Por tanto, no es necesario cambiar la estructura, sino solo entrenar el modelo con diferentes datos de entrada y salida.

- Esta estrategia permite aprovechar una arquitectura bien probada y evita sobrecomplicar el código, enfocándose en cómo se entrena el modelo más que en cómo se diseña la red en sí.

---

In [ ]:
def generar_ruido(x_train, x_test, noise_factor=0.5):
    """
    Añade ruido gaussiano a las imágenes y asegura que los valores queden en [0, 1].

    Args:
        x_train (ndarray): Datos de entrenamiento.
        x_test (ndarray): Datos de prueba.
        noise_factor (float, opcional): Intensidad del ruido. Default 0.5.

    Returns:
        tuple: (x_train_noisy, x_test_noisy) con las imágenes ruidosas.
    """
    x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
    x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
    # Recortar valores al rango [0, 1]
    x_train_noisy = np.clip(x_train_noisy, 0., 1.)
    x_test_noisy = np.clip(x_test_noisy, 0., 1.)
    return x_train_noisy, x_test_noisy

def entrenar_denoising_autoencoder(autoencoder, x_train_noisy, x_train, x_test_noisy, x_test, epochs=20, batch_size=256):
    """
    Entrena un autoencoder para eliminar ruido, usando imágenes ruidosas como entrada
    y limpias como salida.

    Args:
        autoencoder (Model): Modelo base del autoencoder.
        x_train_noisy (ndarray): Datos de entrenamiento ruidosos.
        x_train (ndarray): Datos de entrenamiento limpios.
        x_test_noisy (ndarray): Datos de validación ruidosos.
        x_test (ndarray): Datos de validación limpios.
        epochs (int, opcional): Número de épocas. Default 20.
        batch_size (int, opcional): Tamaño de lote. Default 256.

    Returns:
        tuple: (denoising_autoencoder, history) con el modelo entrenado y su historial.
    """
    denoising_autoencoder = Model(autoencoder.input, autoencoder.output)  # misma arquitectura
    denoising_autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')
    history = denoising_autoencoder.fit(
        x_train_noisy, x_train,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=True,
        validation_data=(x_test_noisy, x_test),
        verbose=0
    )
    return denoising_autoencoder, history

**Bloque 4:** Comparación de modelos

- **`graficar_perdidas_comparadas()`** 
Visualizala evolución comparativa de las pérdidas de entrenamiento y validación de un autoencoder básico y un denoising autoencoder.

- **`mostrar_reconstrucciones_combinadas()`** 
Muestra una figura con las imágenes originales, las reconstrucciones del autoencoder básico, las imágenes ruidosas y las reconstrucciones del denoising autoencoder para comparar visualmente su desempeño.

- **`comparar_perdidas()`** 
Muestra una tabla comparativa de las pérdidas de entrenamiento y validación de ambos modelos.

---

In [ ]:
def graficar_perdidas_comparadas(history_A, history_B):
    """
    Grafica en un solo gráfico la comparación de las pérdidas de entrenamiento y validación 
    de dos modelos: el autoencoder básico y el denoising autoencoder.

    Args:
        history_A (History): Objeto History retornado por el entrenamiento del autoencoder básico.
        history_B (History): Objeto History retornado por el entrenamiento del denoising autoencoder.

    La gráfica incluye cuatro curvas:
        - Pérdida de entrenamiento del autoencoder básico.
        - Pérdida de validación del autoencoder básico.
        - Pérdida de entrenamiento del denoising autoencoder.
        - Pérdida de validación del denoising autoencoder.
    """
    plt.plot(history_A.history['loss'], label='Entrenamiento Autoencoder')
    plt.plot(history_A.history['val_loss'], label='Validación Autoencoder')
    plt.plot(history_B.history['loss'], label='Entrenamiento Denoising')
    plt.plot(history_B.history['val_loss'], label='Validación Denoising')
    plt.title("Comparación de pérdidas de reconstrucción")
    plt.xlabel("Épocas")
    plt.ylabel("Binary Crossentropy")
    plt.legend()
    plt.grid(True)
    plt.show()


def mostrar_reconstrucciones_combinadas(autoencoder, denoising_autoencoder, x_test, x_test_noisy, n=10):
    """
    Muestra una figura con reconstrucciones comparativas de dos modelos: el autoencoder básico y 
    el denoising autoencoder, junto con las imágenes originales y ruidosas.

    La visualización consta de 4 filas y n columnas, donde cada fila representa:
        1. Imágenes originales (limpias).
        2. Reconstrucciones del autoencoder básico.
        3. Imágenes con ruido añadido.
        4. Reconstrucciones del denoising autoencoder.

    Args:
        autoencoder (Model): Modelo Keras entrenado del autoencoder básico.
        denoising_autoencoder (Model): Modelo Keras entrenado del denoising autoencoder.
        x_test (ndarray): Imágenes originales de prueba (sin ruido).
        x_test_noisy (ndarray): Imágenes de prueba con ruido añadido.
        n (int, opcional): Número de imágenes a mostrar por fila. Por defecto 10.

    La figura utiliza GridSpec para reservar una columna adicional a la izquierda, donde se colocan
    etiquetas que identifican cada fila (por ejemplo, "Originales", "Reconstrucción Autoencoder", etc.).
    """
    decoded_basic = autoencoder.predict(x_test)
    decoded_denoise = denoising_autoencoder.predict(x_test_noisy)

    etiquetas_filas = ["Originales", "Reconstrucción\nAutoencoder", "Ruidosas", "Reconstrucción\nDenoising"]

    fig = plt.figure(figsize=(2 + n*2, 8))  # un poco de ancho extra para etiquetas

    gs = GridSpec(4, n+1, width_ratios=[1.5, *([2]*n)], wspace=0.05, hspace=0.1)

    for fila in range(4):
        # Etiqueta en la primera columna de la fila, centrada verticalmente
        ax_label = fig.add_subplot(gs[fila, 0])
        ax_label.text(0.5, 0.5, etiquetas_filas[fila], fontsize=12, ha='center', va='center', rotation=0)
        ax_label.axis('off')

        for col in range(n):
            ax = fig.add_subplot(gs[fila, col+1])
            if fila == 0:
                ax.imshow(x_test[col].reshape(28, 28), cmap='gray')
            elif fila == 1:
                ax.imshow(decoded_basic[col].reshape(28, 28), cmap='gray')
            elif fila == 2:
                ax.imshow(x_test_noisy[col].reshape(28, 28), cmap='gray')
            else:  # fila == 3
                ax.imshow(decoded_denoise[col].reshape(28, 28), cmap='gray')

            ax.axis('off')

    plt.show()

def comparar_perdidas(history_A, history_B):
    """
    Crea y muestra una tabla comparativa de pérdidas y validaciones entre
    un autoencoder básico y un denoising autoencoder.

    Args:
        history_A (History): Historial del entrenamiento del autoencoder básico.
        history_B (History): Historial del entrenamiento del denoising autoencoder.
    """
    """
    Crea y muestra una tabla comparativa de pérdidas y validaciones
    entre un autoencoder básico y un denoising autoencoder.

    Parámetros:
        history_A: objeto History del entrenamiento del autoencoder básico
        history_B: objeto History del entrenamiento del denoising autoencoder
    """
    loss_A = history_A.history['loss']
    val_loss_A = history_A.history['val_loss']
    loss_B = history_B.history['loss']
    val_loss_B = history_B.history['val_loss']

    df = pd.DataFrame({
        'Epoch': list(range(1, len(loss_A) + 1)),
        'Loss_Autoencoder': loss_A,
        'Val_Loss_Autoencoder': val_loss_A,
        'Loss_Denoising': loss_B,
        'Val_Loss_Denoising': val_loss_B
    })

    display(df.style.hide(axis='index'))

**Bloque 4:** Función de ejecución main.

- **`main()`** 
Función principal que ejecuta el flujo completo de entrenamiento, evaluación y comparación de un autoencoder básico y un denoising autoencoder sobre MNIST.

---

In [ ]:
def main():
    """
    Función que ejecuta el flujo completo: carga y preprocesa datos, construye y entrena 
    el autoencoder básico, genera datos ruidosos, entrena el denoising autoencoder,
    grafica las pérdidas y visualiza reconstrucciones, y finalmente compara las pérdidas
    de ambos modelos en una tabla.
    """
    # Carga y preprocesamiento
    print("Cargando datos")
    x_train, x_test = cargar_datos()

    # Construcción y entrenamiento del autoencoder básico
    print("Construyendo y entrenando Autoencoder básico")
    autoencoder = construir_autoencoder(input_dim=x_train.shape[1])
    history_A = entrenar_autoencoder(autoencoder, x_train, x_test)

    # Generación de ruido y entrenamiento del denoising autoencoder
    print("Generando ruido y entrenando Denoising Autoencoder")
    x_train_noisy, x_test_noisy = generar_ruido(x_train, x_test)
    denoising_autoencoder, history_B = entrenar_denoising_autoencoder(
        autoencoder, x_train_noisy, x_train, x_test_noisy, x_test
    )

    # Gráficas de pérdidas comparadas entre ambos modelos
    print("="*40)
    print("Gráficas de pérdidas")
    print("="*40)
    graficar_perdidas_comparadas(history_A, history_B)

    # Reconstrucciones comparadas entre ambos modelos
    print("="*40)
    print("Reconstrucciones comparadas")
    print("="*40)
    mostrar_reconstrucciones_combinadas(autoencoder, denoising_autoencoder, x_test, x_test_noisy, n=10)

    # Comparación de pérdidas entre ambos modelos
    print("="*40)
    print("Comparación de pérdidas")
    print("="*40)
    comparar_perdidas(history_A, history_B)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

## 1. Evaluación de resultados entre los dos modelos
- La comparación de las pérdidas de entrenamiento y validación muestra que el autoencoder básico presenta consistentemente valores más bajos que el denoising autoencoder. Esto indica que, en términos numéricos, el modelo básico es más eficiente reconstruyendo imágenes limpias sin ruido, logrando una mayor precisión en la reconstrucción.

- Por otro lado, el denoising autoencoder presenta pérdidas más altas, lo cual es esperable debido a la mayor complejidad de su tarea: debe reconstruir imágenes limpias a partir de versiones ruidosas. Esta dificultad adicional influye en que el modelo no alcance una pérdida tan baja como el autoencoder básico.

- En cuanto a la evaluación visual, el autoencoder básico tiende a producir reconstrucciones casi idénticas a las imágenes originales, si bien no son exactamente iguales, logran mostrar una alta fidelidad en la reproducción de los detalles. Mientras tanto, el denoising autoencoder cumple con su función de eliminar ruido de las imágenes, pero a menudo puede suavizar detalles finos o introducir pequeñas distorsiones. La calidad visual del denoising depende en gran medida de la cantidad de ruido presente y de la capacidad del modelo para generalizar y eliminar eficazmente las imperfecciones sin degradar la información relevante, en este caso, las reconstrucciones aunque buenas, no alcanzaron el mismo nivel que las reconstruidas cono el autoencoder básico.

- En resumen:
    - El autoencoder básico presenta consistentemente pérdidas de entrenamiento y validación más bajas que el denoising autoencoder.
    - Esto indica que, en términos numéricos, el modelo básico reconstruye mejor las imágenes limpias sin ruido.
    - El denoising autoencoder tiene pérdidas más altas, lo cual es esperable dado que su tarea es más compleja: reconstruir imágenes limpias a partir de versiones ruidosas.

## 2. Reflexión sobre el potencial uso de autoencoders y denoising autoencoders
- Las técnicas de autoencoders, y particularmente los denoising autoencoders, tienen un enorme potencial en campos como medicina, seguridad e industria: 

    - En medicina, estos modelos pueden utilizarse para mejorar la calidad de imágenes médicas (como resonancias magnéticas, tomografías o rayos X) eliminando ruido o artefactos que dificultan el diagnóstico. Esto puede ayudar a los profesionales a detectar anomalías con mayor precisión y a reducir falsos positivos o negativos, lo que impacta directamente en la calidad del tratamiento y pronóstico.

    - En seguridad, los autoencoders pueden emplearse para la detección de anomalías en señales o datos, como identificar accesos no autorizados, fraudes o comportamientos irregulares en sistemas complejos. La capacidad de reconstruir datos esperados y detectar desviaciones hace que sean una herramienta valiosa para proteger infraestructuras críticas y datos sensibles.

    - En la industria, los autoencoders pueden mejorar el mantenimiento predictivo, analizando señales de sensores en maquinaria para detectar fallos incipientes. La eliminación de ruido en estas señales permite una monitorización más fiable y la anticipación de problemas, evitando costosas interrupciones y optimizando recursos.

- En resumen, el uso de autoencoders y denoising autoencoders aporta un equilibrio entre la reducción de dimensionalidad, limpieza de datos y detección de patrones, convirtiéndolos en herramientas fundamentales para mejorar procesos, aumentar la seguridad y apoyar la toma de decisiones en entornos cada vez más complejos y automatizados.

---